# Введение

В данном проекте реализована задача классификации изображений с двумя классами: кошки и собаки.  

Целью данного проекта является создание и обучение нейросетевой модели, способной с высокой точностью различать изображения кошек и собак. Для решения этой задачи используется предобученная модель MobileNet, которая была модифицирована под данную задачу с учетом специфики предоставленного набора данных

Для обучения модели используется популярный датасет "Кошки и Собаки" (Cats and Dogs), который содержит тысячи изображений двух классов: кошек и собак

# Импорт библиотек

In [ ]:
import os
import shutil
from keras import layers
from keras import models
from keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNet
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input

# Загрузка и распаковка датасета

In [ ]:
!wget https://storage.yandexcloud.net/academy.ai/cat-and-dog.zip
!unzip -qo "cat-and-dog" -d ./temp

IMAGE_PATH = './temp/training_set/training_set/'
BASE_DIR = './dataset/'

# Train-val-test

In [ ]:
CLASS_LIST = sorted(os.listdir(IMAGE_PATH))
CLASS_COUNT = len(CLASS_LIST)

if os.path.exists(BASE_DIR):
    shutil.rmtree(BASE_DIR)

os.mkdir(BASE_DIR)
train_dir = os.path.join(BASE_DIR, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(BASE_DIR, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(BASE_DIR, 'test')
os.mkdir(test_dir)

# Каталог для датасета

In [ ]:
def create_dataset(img_path, new_path, class_name, start_index, end_index):
    src_path = os.path.join(img_path, class_name)
    dst_path = os.path.join(new_path, class_name)
    class_files = os.listdir(src_path)
    os.mkdir(dst_path)
    for fname in class_files[start_index:end_index]:
        src = os.path.join(src_path, fname)
        dst = os.path.join(dst_path, fname)
        shutil.copyfile(src, dst)

In [ ]:
for class_label in range(CLASS_COUNT):
    class_name = CLASS_LIST[class_label]
    create_dataset(IMAGE_PATH, train_dir, class_name, 0, 2000)
    create_dataset(IMAGE_PATH, validation_dir, class_name, 2000, 3000)
    create_dataset(IMAGE_PATH, test_dir, class_name, 3000, 4000)

# Генерация данных

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical'
)

# Константы

In [ ]:
IMG_WIDTH = 150
IMG_HEIGHT = 150
NUM_CLASSES = CLASS_COUNT

# Создание кастомной модели на основе архитектуры MobileNet

In [ ]:
def model_maker():
    base_model = MobileNet(include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

    for layer in base_model.layers[:]:
        layer.trainable = False

    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)

    return Model(inputs=input, outputs=predictions)

model = model_maker()
model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50
)

# Оценка

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps=50)
print('Точность на контрольной выборке:', test_acc)